In [90]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
import keras_tuner as kt
from tensorflow.keras import optimizers

In [16]:
X = np.loadtxt('feature_data.csv', delimiter=',')
y = np.loadtxt('class_data.csv', delimiter=',')

print(X)
print(np.count_nonzero(y))
print(np.size(y))
print(753/1403)

[[ 1.92766025  0.12958038  0.0280275   0.03613106  0.2809    ]
 [ 0.31014981  0.08610681  0.03454437  0.06106653  0.3229    ]
 [ 0.18029989  0.19139025  0.02871548  0.04061004  0.393     ]
 ...
 [ 1.07716387  0.17174854  0.01938974  0.05568655  0.4301    ]
 [-0.08929531  0.14845486 -0.01286206  0.08363529  0.5746    ]
 [ 1.05222792  0.16738197  0.02628362  0.07058851  0.3659    ]]
753
1403
0.5367070563079116


In [61]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.30,)#random_state=46)

def accuracy(y_pred, y_true):
    total = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_test[i]: total +=1 
    return total / len(y_pred)
#print("Accuracy: {}".format(total / len(y_pred)))

In [85]:
for i in range(50,1500, 50):
    tree_model = RandomForestClassifier()
    y_pred = tree_model.fit(X_train, y_train).predict(X_test)
    print("({}) Accuracy: {}".format(i, accuracy(y_pred, y_test)))

(50) Accuracy: 0.6342042755344418
(100) Accuracy: 0.6365795724465558
(150) Accuracy: 0.6437054631828979
(200) Accuracy: 0.6365795724465558
(250) Accuracy: 0.6389548693586699
(300) Accuracy: 0.6603325415676959
(350) Accuracy: 0.6389548693586699
(400) Accuracy: 0.6460807600950119
(450) Accuracy: 0.6413301662707839
(500) Accuracy: 0.6555819477434679
(550) Accuracy: 0.6389548693586699
(600) Accuracy: 0.6294536817102138
(650) Accuracy: 0.6508313539192399
(700) Accuracy: 0.6437054631828979
(750) Accuracy: 0.6532066508313539
(800) Accuracy: 0.6389548693586699
(850) Accuracy: 0.6413301662707839
(900) Accuracy: 0.66270783847981
(950) Accuracy: 0.6555819477434679
(1000) Accuracy: 0.6603325415676959
(1050) Accuracy: 0.6484560570071259
(1100) Accuracy: 0.6508313539192399
(1150) Accuracy: 0.6508313539192399
(1200) Accuracy: 0.6532066508313539
(1250) Accuracy: 0.6508313539192399
(1300) Accuracy: 0.6460807600950119
(1350) Accuracy: 0.6508313539192399
(1400) Accuracy: 0.6460807600950119
(1450) Accurac

In [87]:
tree_model = RandomForestClassifier(n_estimators=1000)
y_pred = tree_model.fit(X_train, y_train).predict(X_test)
metrics.roc_curve(y_test, y_pred)
#print("Accuracy: {}".format(accuracy(y_pred, y_test)))

(array([0.       , 0.3968254, 1.       ]),
 array([0.        , 0.66810345, 1.        ]),
 array([2., 1., 0.]))

In [83]:
#performs best when random state is 46

#simple test example
for i in range(50,1500, 50):
    rf = AdaBoostClassifier(n_estimators=i)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    print("({}) Accuracy: {}".format(i, accuracy(y_pred, y_test)))

(50) Accuracy: 0.6342042755344418
(100) Accuracy: 0.6365795724465558
(150) Accuracy: 0.6603325415676959
(200) Accuracy: 0.6603325415676959
(250) Accuracy: 0.669833729216152
(300) Accuracy: 0.66270783847981
(350) Accuracy: 0.66270783847981
(400) Accuracy: 0.66270783847981
(450) Accuracy: 0.6508313539192399
(500) Accuracy: 0.672209026128266
(550) Accuracy: 0.669833729216152
(600) Accuracy: 0.66270783847981
(650) Accuracy: 0.6555819477434679
(700) Accuracy: 0.6508313539192399
(750) Accuracy: 0.6579572446555819
(800) Accuracy: 0.6579572446555819
(850) Accuracy: 0.6603325415676959
(900) Accuracy: 0.667458432304038
(950) Accuracy: 0.672209026128266
(1000) Accuracy: 0.665083135391924
(1050) Accuracy: 0.669833729216152
(1100) Accuracy: 0.667458432304038
(1150) Accuracy: 0.66270783847981
(1200) Accuracy: 0.665083135391924
(1250) Accuracy: 0.665083135391924
(1300) Accuracy: 0.669833729216152
(1350) Accuracy: 0.6603325415676959
(1400) Accuracy: 0.6579572446555819
(1450) Accuracy: 0.65795724465558

In [88]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Accuracy: {}".format(accuracy(y_pred, y_test)))

Accuracy: 0.4489311163895487


In [127]:
model = Sequential()
kh = kt.HyperParameters()
model.add(Dense(units=kh.Int("units", min_value=5, max_value=200), activation='relu', input_dim=5))
model.add(Dense(units=kh.Int("units", min_value=5, max_value=200, step=32), activation=kh.Choice("activation", ["relu", "tanh"])))
model.add(Dense(units=kh.Int("units", min_value=5, max_value=200, step=32), activation=kh.Choice("activation", ["relu", "tanh"])))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=optimizers.RMSprop(), loss='binary_crossentropy', metrics=['accuracy'])

In [128]:
nn_res = model.fit(X_train, y_train, epochs=70, batch_size=20,validation_split=.3)

Epoch 1/70
35/35 [==============================] - 0s 3ms/step - loss: 0.7009 - accuracy: 0.5226 - val_loss: 0.6927 - val_accuracy: 0.5729
Epoch 2/70
35/35 [==============================] - 0s 925us/step - loss: 0.6963 - accuracy: 0.5546 - val_loss: 0.6915 - val_accuracy: 0.5492
Epoch 3/70
35/35 [==============================] - 0s 973us/step - loss: 0.6934 - accuracy: 0.5604 - val_loss: 0.6909 - val_accuracy: 0.5356
Epoch 4/70
35/35 [==============================] - 0s 1ms/step - loss: 0.6911 - accuracy: 0.5488 - val_loss: 0.6907 - val_accuracy: 0.5424
Epoch 5/70
35/35 [==============================] - 0s 913us/step - loss: 0.6896 - accuracy: 0.5560 - val_loss: 0.6908 - val_accuracy: 0.5051
Epoch 6/70
35/35 [==============================] - 0s 983us/step - loss: 0.6884 - accuracy: 0.5531 - val_loss: 0.6913 - val_accuracy: 0.4949
Epoch 7/70
35/35 [==============================] - 0s 936us/step - loss: 0.6875 - accuracy: 0.5400 - val_loss: 0.6918 - val_accuracy: 0.5017
Epoch 8/70

35/35 [==============================] - 0s 888us/step - loss: 0.6678 - accuracy: 0.5997 - val_loss: 0.6776 - val_accuracy: 0.5864
Epoch 59/70
35/35 [==============================] - 0s 912us/step - loss: 0.6679 - accuracy: 0.5953 - val_loss: 0.6792 - val_accuracy: 0.5898
Epoch 60/70
35/35 [==============================] - 0s 949us/step - loss: 0.6675 - accuracy: 0.5983 - val_loss: 0.6787 - val_accuracy: 0.5932
Epoch 61/70
35/35 [==============================] - 0s 916us/step - loss: 0.6671 - accuracy: 0.5953 - val_loss: 0.6782 - val_accuracy: 0.5932
Epoch 62/70
35/35 [==============================] - 0s 1ms/step - loss: 0.6670 - accuracy: 0.6026 - val_loss: 0.6772 - val_accuracy: 0.5898
Epoch 63/70
35/35 [==============================] - 0s 954us/step - loss: 0.6670 - accuracy: 0.5983 - val_loss: 0.6785 - val_accuracy: 0.5966
Epoch 64/70
35/35 [==============================] - 0s 933us/step - loss: 0.6663 - accuracy: 0.5939 - val_loss: 0.6785 - val_accuracy: 0.6034
Epoch 65/70
3

In [129]:
y_pred = model.predict(X_test)
y_pred[ y_pred >= .5] = 1
y_pred[y_pred < .5] = 0
print("Accuracy: {}".format(accuracy(y_pred, y_test)))

Accuracy: 0.6033254156769596
